# find the faces

In [ ]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        cv2.imshow('roi', roi_gray)

        color = (255, 0, 0)
        thickness = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, thickness)

    cv2.imshow('frame', frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 


# Select the image from the folder

In [ ]:
from PyQt5.QtWidgets import QApplication, QFileDialog, QDialog, QVBoxLayout, QLabel, QLineEdit, QPushButton
from PyQt5.QtGui import QPixmap
import os

# create an instance of the QApplication
app = QApplication([])

# show the dialog to get the destination folder name
dialog = QDialog()
dialog.setWindowTitle('Select the images from the folder')
layout = QVBoxLayout()
label = QLabel('Enter the name of the USER:')
layout.addWidget(label)
line_edit = QLineEdit()
layout.addWidget(line_edit)
button = QPushButton('OK')
layout.addWidget(button)
dialog.setLayout(layout)

def on_button_clicked():
    dialog.accept()

button.clicked.connect(on_button_clicked)
result = dialog.exec_()
if result == QDialog.Accepted:
    dest_folder_name = line_edit.text()
else:
    dest_folder_name = None

if dest_folder_name:
    # create the new folder if it doesn't already exist
    dest_folder = os.path.join('images', dest_folder_name)
    if not os.path.exists(dest_folder):
        os.mkdir(dest_folder)

    for i in range(3):
        # show the file dialog to select the image file
        file_path, _ = QFileDialog.getOpenFileName(
            caption=f'Select Image File {i+1}',
            filter='Image Files (*.jpg *.jpeg *.png)'
        )

        if file_path:
            # get the file name and extension
            file_name = os.path.basename(file_path)
            file_ext = file_name.split('.')[-1]

            # create the destination file path
            dest_file_path = f'{dest_folder}/{file_name}'

            # load the image using QPixmap
            pixmap = QPixmap(file_path)

            # save the image to the destination folder
            pixmap.save(dest_file_path)

            print(f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            print(f'No image file selected for image {i+1}')
else:
    print('No destination folder selected')

# quit the application
app.quit()


# capture the image from the webcam

In [ ]:
import cv2
import os
from PyQt5.QtWidgets import QApplication, QFileDialog, QDialog, QVBoxLayout, QLabel, QLineEdit, QPushButton

# create an instance of the QApplication
app = QApplication([])

# show the dialog to get the user's name
dialog = QDialog()
dialog.setWindowTitle('Enter User Name')
layout = QVBoxLayout()
label = QLabel('Enter your name:')
layout.addWidget(label)
line_edit = QLineEdit()
layout.addWidget(line_edit)
button = QPushButton('OK')
layout.addWidget(button)
dialog.setLayout(layout)

def on_button_clicked():
    dialog.accept()

button.clicked.connect(on_button_clicked)
result = dialog.exec_()
if result == QDialog.Accepted:
    user_name = line_edit.text()
else:
    user_name = None

if user_name:
    # create the new folder for the user's images
    user_folder = os.path.join('images', user_name)
    if not os.path.exists(user_folder):
        os.mkdir(user_folder)
else:
    print('No user name entered')
    
# Loading classifier
faceCascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

# Capturing real time video stream. 0 for built-in web-cams, 0 or -1 for external web-cams
video_capture = cv2.VideoCapture(0)

# set counter variable to 0
count = 0

# Method to draw boundary around the detected face
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    # Converting image to gray-scale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detecting face in gray-scale image
    faces = classifier.detectMultiScale(gray_img, scaleFactor=scaleFactor, minNeighbors=minNeighbors)
    # Creating rectangle around the face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
    # Returning processed image
    return img

def detect(img, faceCascade):
    # Convert image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect face in the grayscale image
    faces = faceCascade.detectMultiScale(gray_img, scaleFactor=1.2, minNeighbors=5)
    
    # Loop through each face detected and capture face region
    for (x, y, w, h) in faces:
        roi_color = img[y:y+h, x:x+w]
        
        # Increment counter
        global count
        count += 1
        
        # save the face image in the user's folder
        cv2.imwrite(os.path.join(user_folder, f"{count}.jpg"), roi_color)
        
        # Draw boundary around the detected face
        img = draw_boundary(img, faceCascade, 1.1, 10, (255, 0, 0), 'Face')
    
    return img

# Start capturing the video
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Detect faces and capture face region
    frame = detect(frame, faceCascade)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Wait for 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q') or count == 100:
        break

# Release the camera and close all windows
video_capture.release()
cv2.destroyAllWindows()



# Regonize the face

In [5]:
import os
from PIL import Image
import cv2
import numpy as np
import pickle



BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, "images")


face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "-").lower()
            print(label, path)
            if not label in label_ids:
                label_ids[label]= current_id
                current_id +=1
            id_= label_ids[label]
            print(label_ids)
            
            pil_image = Image.open(path).convert("L")
            size = (550,550)
            final_image = pil_image.resize(size,Image.Resampling.LANCZOS)
            image_array = np.array(final_image, "uint8")
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            
            for(x,y,w,h) in faces:
                roi= image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

            
            
with open("pickles/face-labels.pickle", 'wb') as f:
	pickle.dump(label_ids, f)

recognizer.train(x_train, np.array(y_labels))
recognizer.save("recognizers/face-trainner.yml")

anant C:\Users\Hp\face project\images\anant\HD-wallpaper-krishna-feather-pecock-thumbnail.jpg
{'anant': 0}
anant C:\Users\Hp\face project\images\anant\maxresdefault.jpg
{'anant': 0}
anant C:\Users\Hp\face project\images\anant\WIN_20230504_09_39_41_Pro.jpg
{'anant': 0}
asd C:\Users\Hp\face project\images\asd\1bb32b8c-d647-40d2-a7ab-4d5717e40a9d.jpg
{'anant': 0, 'asd': 1}
asd C:\Users\Hp\face project\images\asd\242-2425226_khatu-shyam-baba-photo-sh-khatu-shyam-bhajan.jpg
{'anant': 0, 'asd': 1}
asd C:\Users\Hp\face project\images\asd\6086508.jpg
{'anant': 0, 'asd': 1}
asd C:\Users\Hp\face project\images\asd\HD-wallpaper-krishna-feather-pecock-thumbnail.jpg
{'anant': 0, 'asd': 1}
asd C:\Users\Hp\face project\images\asd\maxresdefault.jpg
{'anant': 0, 'asd': 1}
emilia-clarke C:\Users\Hp\face project\images\emilia-clarke\1.jpg
{'anant': 0, 'asd': 1, 'emilia-clarke': 2}
emilia-clarke C:\Users\Hp\face project\images\emilia-clarke\2.jpg
{'anant': 0, 'asd': 1, 'emilia-clarke': 2}
emilia-clarke C:

# Implement recognizer


In [3]:
import numpy as np
import cv2
import pickle

face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('recognizers/face-trainner.yml')

labels = {"person_name": 1}
with open("pickles/face-labels.pickle", 'rb') as f:
	og_labels = pickle.load(f)
	labels = {v:k for k,v in og_labels.items()}

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        if conf>=4 and conf<=85:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255, 255, 255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

        
        img_item = "7.png"
        cv2.imshow('roi', roi_gray)

        color = (255, 0, 0)
        thickness = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, thickness)

    cv2.imshow('frame', frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 


# Delete the User ID

In [1]:
import os
import shutil
from PyQt5.QtWidgets import QApplication, QDialog, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox

class DeleteFolderDialog(QDialog):
    def __init__(self, folder_path):
        super().__init__()
        self.setWindowTitle("Delete Folder")
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.folder_label = QLabel("Enter the name of the user you want to delete:")
        self.folder_line_edit = QLineEdit(folder_path)
        self.layout.addWidget(self.folder_label)
        self.layout.addWidget(self.folder_line_edit)
        self.delete_button = QPushButton("Delete Folder")
        self.delete_button.clicked.connect(self.delete_folder)
        self.layout.addWidget(self.delete_button)
        self.exit_button = QPushButton("Exit")
        self.exit_button.clicked.connect(self.close)
        self.layout.addWidget(self.exit_button)

    def delete_folder(self):
        folder_name = self.folder_line_edit.text()
        folder_path = os.path.join(os.getcwd(), "images", folder_name)
        if os.path.exists(folder_path):
            button_reply = QMessageBox.question(self, "Delete user", "Are you sure you want to delete this user ?",
                                                 QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
            if button_reply == QMessageBox.Yes:
                try:
                    # Use os.rmdir to delete an empty directory
                    os.rmdir(folder_path)
                except OSError:
                    # Use shutil.rmtree to delete a non-empty directory
                    shutil.rmtree(folder_path)
                QMessageBox.information(self, "Delete user", " deleted successfully!")
        else:
            QMessageBox.warning(self, "Delete Folder", "Folder not found. Please check the folder name and try again.")


if __name__ == '__main__':
    app = QApplication([])
    dialog = DeleteFolderDialog("")
    dialog.exec_()


# Restart the Program

In [1]:
import os
from PyQt5.QtWidgets import QApplication, QMessageBox

class DeleteFileDialog(QMessageBox):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Restart")
        self.setText("Are you sure you want to RESTART the Program")
        self.setStandardButtons(QMessageBox.Yes | QMessageBox.No)
        self.setDefaultButton(QMessageBox.No)
        self.buttonClicked.connect(self.delete_file)

    def delete_file(self, button):
        if button.text() == "&Yes":
            file_path = os.path.join(os.getcwd(), "recognizers", "face-trainner.yml")
            try:
                os.remove(file_path)
                self.information(self, "Restart", "You program restart successfull")
            except OSError:
                self.warning(self, "Restart", "Error. Please try again.")
        else:
            self.close()


if __name__ == '__main__':
    app = QApplication([])
    dialog = DeleteFileDialog()
    dialog.exec_()


In [4]:
BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, "images")
print(image_dir)

C:\Users\Hp\face project\images
